** This file is to find the gym locations from Yelp **
1. find the number of the gyms
2. get the locations of the gyms
3. output the coordinates of each gym to a csv file 

In [19]:
from config import API_KEY
import requests
import sample
import pandas as pd

In [20]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses'

In [21]:
locations = ["Austin,TX"]
term = "gyms"
open_at = 1528961400  #6/14/2018 2:30 a.m. Austin time
limit = 50
final_file = "Austin.csv"

In [22]:
gym = []
lat = []
long = []

# get the locations of 24-hr gyms
for location in locations:
    params={
        "term" : term,
        "location" : location,
        "open_at" : open_at,
        "limit" : limit,
        "offset" : 0
    }
    
    response = sample.request(API_HOST, SEARCH_PATH, API_KEY, params)
    total_gym = response['total']
    
    
    print(f"{total_gym} non-24-hr gyms in total")
    
    offset_time = int(total_gym/limit)
    
    
    for i in range(limit):
        print(i, ":", response['businesses'][i]['name'], "-", response['businesses'][i]['coordinates'])
        gym.append(response['businesses'][i]['name'])
        lat.append(response['businesses'][i]['coordinates']['latitude'])
        long.append(response['businesses'][i]['coordinates']['longitude'])
    
    if (offset_time != 0):
        for run in range(offset_time):
            offset = (run+1)*limit
            params={
                "term" : term,
                "location" : location,
                "open_at" : open_at,
                "limit" : limit,
                "offset" : offset
            }
            print ("offset:",offset)
            
            response = sample.request(API_HOST, SEARCH_PATH, API_KEY, params)
            total_gym = response['total']
            print(f"{total_gym-offset} non-24-hr gyms left")
            
            try:
                for i in range(limit):
                    gym.append(response['businesses'][i]['name'])
                    lat.append(response['businesses'][i]['coordinates']['latitude'])
                    long.append(response['businesses'][i]['coordinates']['longitude'])
                    print(i+offset, ":", response['businesses'][i]['name'], "-", response['businesses'][i]['coordinates'])
                    
            
            except IndexError:
                break

        
  

Querying https://api.yelp.com/v3/businesses/search ...
55 non-24-hr gyms in total
0 : Life Time Fitness - {'latitude': 30.215458, 'longitude': -97.840105}
1 : Anytime Fitness - {'latitude': 30.4171, 'longitude': -97.703761}
2 : Anytime Fitness - {'latitude': 30.30057, 'longitude': -97.73923}
3 : Anytime Fitness - {'latitude': 30.35508, 'longitude': -97.73323}
4 : 24 Hour Fitness - Hancock Center - {'latitude': 30.301434809559, 'longitude': -97.72055964814}
5 : Anytime Fitness - {'latitude': 30.4018662998415, 'longitude': -97.8534534505613}
6 : Anytime Fitness - {'latitude': 30.456559, 'longitude': -97.76716}
7 : Bee Cave Fitness - {'latitude': 30.33811, 'longitude': -97.97113}
8 : Life Time Fitness - {'latitude': 30.472509, 'longitude': -97.774084}
9 : Planet Fitness - Austin - E. Riverside - {'latitude': 30.23452, 'longitude': -97.721}
10 : Anytime Fitness - {'latitude': 30.372375, 'longitude': -97.802626}
11 : 24 Hour Fitness - William Cannon - {'latitude': 30.2219745482289, 'longitu

In [23]:
total_gym

55

In [16]:
len(long)

113

In [17]:
gym_loc = pd.DataFrame({
    "Name" : gym,
    "Lat" : lat,
    "Long" : long
})

In [18]:
gym_loc.to_csv(final_file, index=False)